<a href="https://colab.research.google.com/github/ishubham4/MIS_REPORT/blob/main/Report.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%pip install openpyxl
import pandas as pd 
import numpy as np

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#Anlaysing Company X- Order report
df1 = pd.read_excel("/content/drive/MyDrive/Cointab Data Analyst - Challenge/Company X - Order Report.xlsx")
df1.ExternOrderNo.unique() # here we got to know that there are total 124 unique Order ID 
df1.ExternOrderNo.nunique() 
df1.rename(columns = {'ExternOrderNo':'Order ID'}, inplace = True) # Also renaming ExternOrderNo to Order ID
df1

,Order ID,SKU,Order Qty
0,2001827036,8904223818706,1.0
1,2001827036,8904223819093,1.0
2,2001827036,8904223819109,1.0
3,2001827036,8904223818430,1.0
4,2001827036,8904223819277,1.0
...,...,...,...
395,2001806229,8904223818942,1.0
396,2001806229,8904223818850,1.0
397,2001806226,8904223818850,2.0
398,2001806210,8904223816214,1.0


In [ ]:
df2= pd.read_excel("/content/drive/MyDrive/Cointab Data Analyst - Challenge/Company X - SKU Master.xlsx")
df2

df2.SKU.unique() # checking for unique values
df2.SKU.nunique() # counting the uniques values

# droping the duplicates values 

df3 =df2.drop_duplicates() #new dataframe without the duplicates values
df3


,SKU,Weight (g)
0,8904223815682,210
1,8904223815859,165
2,8904223815866,113
3,8904223815873,65
4,8904223816214,120
...,...,...
61,8904223819505,210
62,8904223819499,210
63,8904223819512,210
64,8904223819543,300


In [ ]:
Order = pd.merge(df1, df3, on="SKU") # Merging both Order Report dataset anf SKU Master data 
Order['Actual Weight(g)']=Order["Order Qty"]* Order['Weight (g)'] # To get the actual weight of the Order ,here we have multiplied Order Qty with Weight to get the Actual Weight
Order

Xcompany=Order.groupby('Order ID')["Actual Weight(g)"].sum().rename("Total weight as per X (G)").reset_index() # Grouping on the basis of Order ID  and and adding new column as Total Weight as per X(g)
Xcompany["Total weight as per X (KG)"] = Xcompany["Total weight as per X (G)"] / 1000  # Adding another column as Total weight as per X(kg) in existing Dataframe
CompanyReport =Xcompany.round({"Total weight as per X (KG)":1}) #Round to decimal 1
filters = [
   (CompanyReport["Total weight as per X (KG)"] <= 0.5),                               #Setting multiple conditions according to the data as The courier company calculates weight in slabs of 0.5 KG multiples
   (CompanyReport["Total weight as per X (KG)"] > 0.5) & (CompanyReport["Total weight as per X (KG)"] <=1),
   (CompanyReport["Total weight as per X (KG)"] > 1) & (CompanyReport["Total weight as per X (KG)"] <=1.5),
   (CompanyReport["Total weight as per X (KG)"] > 1.5) & (CompanyReport["Total weight as per X (KG)"] <=2),
   (CompanyReport["Total weight as per X (KG)"] > 2) & (CompanyReport["Total weight as per X (KG)"] <=2.5),
   (CompanyReport["Total weight as per X (KG)"] > 2.5) & (CompanyReport["Total weight as per X (KG)"] <=3),
   (CompanyReport["Total weight as per X (KG)"] > 3) & (CompanyReport["Total weight as per X (KG)"] <=3.5)
]
values = [0.5,1,1.5,2,2.5,3,3.5]
CompanyReport["Weight slab as per X (KG)"] = np.select(filters, values) # Applying those conditions and adding a new column Weight slab as per X (KG)
CompanyReport   #X company report



,Order ID,Total weight as per X (G),Total weight as per X (KG),Weight slab as per X (KG)
0,2001806210,220.0,0.2,0.5
1,2001806226,480.0,0.5,0.5
2,2001806229,500.0,0.5,0.5
3,2001806232,1302.0,1.3,1.5
4,2001806233,245.0,0.2,0.5
...,...,...,...,...
119,2001821995,477.0,0.5,0.5
120,2001822466,1376.0,1.4,1.5
121,2001823564,672.0,0.7,1.0
122,2001825261,1557.0,1.6,2.0


In [ ]:
CourierReport = pd.read_excel("/content/drive/MyDrive/Cointab Data Analyst - Challenge/Courier Company - Invoice.xlsx")
CourierReport

Report = pd.merge(CompanyReport, CourierReport, on="Order ID") # Merging the Courier Company Report and X Company Report
Report
filters = [
   (Report["Charged Weight"] <= 0.5),              #Setting multiple conditions according to the data as The courier company calculates weight in slabs of 0.5 KG multiples
   ((Report["Charged Weight"] > 0.5) & (Report["Charged Weight"] <=1)),
   (Report["Charged Weight"] > 1) & (Report["Charged Weight"] <=1.5),
   (Report["Charged Weight"] > 1.5) & (Report["Charged Weight"] <=2),
   (Report["Charged Weight"] > 2) & (Report["Charged Weight"] <=2.5),
   (Report["Charged Weight"] > 2.5) & (Report["Charged Weight"] <=3),
   (Report["Charged Weight"] > 3) & (Report["Charged Weight"] <=3.5),
   (Report["Charged Weight"] > 3.5) & (Report["Charged Weight"] <=4),
   (Report["Charged Weight"] > 4) & (Report["Charged Weight"] <=4.5)

]
values = [0.5,1,1.5,2,2.5,3,3.5,4,4.5]
Report["Weight slab charged by Courier Company (KG))"] = np.select(filters, values) # Applying those conditions and adding a new column Weight slab as per X (KG)
Report.rename(columns = {'Charged Weight':'Total weight as per Courier Company (KG)','Zone': "Delivery Zone charged by Courier Company","Billing Amount (Rs.)": "Charges Billed by Courier Company (Rs.)",'Warehouse Pincode':'Warehouse Pincode as per Courier Company','Customer Pincode':'Customer Pincode as per Courier Company'}, inplace = True) #Also renaming columns 
Report

,Order ID,Total weight as per X (G),Total weight as per X (KG),Weight slab as per X (KG),AWB Code,Total weight as per Courier Company (KG),Warehouse Pincode as per Courier Company,Customer Pincode as per Courier Company,Delivery Zone charged by Courier Company,Type of Shipment,Charges Billed by Courier Company (Rs.),Weight slab charged by Courier Company (KG))
0,2001806210,220.0,0.2,0.5,1091117221940,2.92,121003,140604,b,Forward charges,174.5,3.0
1,2001806226,480.0,0.5,0.5,1091117222065,0.68,121003,723146,d,Forward charges,90.2,1.0
2,2001806229,500.0,0.5,0.5,1091117222080,0.71,121003,421204,d,Forward charges,90.2,1.0
3,2001806232,1302.0,1.3,1.5,1091117222124,1.30,121003,507101,d,Forward charges,135.0,1.5
4,2001806233,245.0,0.2,0.5,1091117222135,0.78,121003,263139,b,Forward charges,61.3,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
119,2001821995,477.0,0.5,0.5,1091121183730,0.50,121003,342008,d,Forward charges,45.4,0.5
120,2001822466,1376.0,1.4,1.5,1091121305541,1.10,121003,342301,d,Forward charges,135.0,1.5
121,2001823564,672.0,0.7,1.0,1091121666133,0.70,121003,492001,d,Forward and RTO charges,172.8,1.0
122,2001825261,1557.0,1.6,2.0,1091121981575,1.60,121003,517128,d,Forward and RTO charges,345.0,2.0


In [ ]:
Pincode = pd.read_excel("/content/drive/MyDrive/Cointab Data Analyst - Challenge/Company X - Pincode Zones.xlsx") # Reading thhe Pincode Zones data to map with the OrderID of X company Report
CourierReport
XOrderPincode =CourierReport.drop(['AWB Code', 'Charged Weight',"Zone","Type of Shipment","Billing Amount (Rs.)"], axis=1) # Dropping the unnecessary data from the Courier Company Report and adding that to new df of XOrderPincode
XOrderPincode
XCompanyPincodeZone = pd.merge(XOrderPincode,Pincode, how='inner', left_index=True,    #Merging the new df created XOrderPicode and Pincode to get the XCompanyPincodeZone 
                  right_index=True)                
XCompanyPincodeZone = XCompanyPincodeZone.drop(['Warehouse Pincode_x', 'Customer Pincode_x'], axis=1)# Dropping the duplicates cloumns from XCompanyPincodeZone 

XCompanyPincodeZone.rename(columns = {'Warehouse Pincode_y':'Warehouse Pincode as per X','Customer Pincode_y':'Customer Pincode as per X', 'Zone':'Delivery Zone as per X'}, inplace = True)
XCompanyPincodeZone # this is data of OrderID where the order need to be delivered according to the X company

,Order ID,Warehouse Pincode as per X,Customer Pincode as per X,Delivery Zone as per X
0,2001806232,121003,507101,d
1,2001806273,121003,486886,d
2,2001806408,121003,532484,d
3,2001806458,121003,143001,b
4,2001807012,121003,515591,d
...,...,...,...,...
119,2001812941,121003,325207,b
120,2001809383,121003,303702,b
121,2001820978,121003,313301,b
122,2001811475,121003,173212,e


In [ ]:
ExpectedResult = pd.merge(Report,XCompanyPincodeZone, on="Order ID") #Merging Report of the Courier Company Report and XCompanyPincodeZone DataSet
ExpectedResult 

,Order ID,Total weight as per X (G),Total weight as per X (KG),Weight slab as per X (KG),AWB Code,Total weight as per Courier Company (KG),Warehouse Pincode as per Courier Company,Customer Pincode as per Courier Company,Delivery Zone charged by Courier Company,Type of Shipment,Charges Billed by Courier Company (Rs.),Weight slab charged by Courier Company (KG)),Warehouse Pincode as per X,Customer Pincode as per X,Delivery Zone as per X
0,2001806210,220.0,0.2,0.5,1091117221940,2.92,121003,140604,b,Forward charges,174.5,3.0,121003,140604,b
1,2001806226,480.0,0.5,0.5,1091117222065,0.68,121003,723146,d,Forward charges,90.2,1.0,121003,723146,d
2,2001806229,500.0,0.5,0.5,1091117222080,0.71,121003,421204,d,Forward charges,90.2,1.0,121003,421204,d
3,2001806232,1302.0,1.3,1.5,1091117222124,1.30,121003,507101,d,Forward charges,135.0,1.5,121003,507101,d
4,2001806233,245.0,0.2,0.5,1091117222135,0.78,121003,263139,b,Forward charges,61.3,1.0,121003,263139,b
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119,2001821995,477.0,0.5,0.5,1091121183730,0.50,121003,342008,d,Forward charges,45.4,0.5,121003,342008,b
120,2001822466,1376.0,1.4,1.5,1091121305541,1.10,121003,342301,d,Forward charges,135.0,1.5,121003,342301,b
121,2001823564,672.0,0.7,1.0,1091121666133,0.70,121003,492001,d,Forward and RTO charges,172.8,1.0,121003,492001,d
122,2001825261,1557.0,1.6,2.0,1091121981575,1.60,121003,517128,d,Forward and RTO charges,345.0,2.0,121003,517128,d


In [ ]:
Rates = pd.read_excel("/content/drive/MyDrive/Cointab Data Analyst - Challenge/Courier Company - Rates.xlsx")
Rates  

,fwd_a_fixed,fwd_a_additional,fwd_b_fixed,fwd_b_additional,fwd_c_fixed,fwd_c_additional,fwd_d_fixed,fwd_d_additional,fwd_e_fixed,fwd_e_additional,rto_a_fixed,rto_a_additional,rto_b_fixed,rto_b_additional,rto_c_fixed,rto_c_additional,rto_d_fixed,rto_d_additional,rto_e_fixed,rto_e_additional
0,29.5,23.6,33,28.3,40.1,38.9,45.4,44.8,56.6,55.5,13.6,23.6,20.5,28.3,31.9,38.9,41.3,44.8,50.7,55.5


In [ ]:
def charge(df):
  if df["Delivery Zone as per X"] =="d" and df["Type of Shipment"] =="Forward charges":
    if df['Weight slab as per X (KG)']<=0.5:
      price = Rates['fwd_d_fixed'].tolist()[0]
    else:
      price = Rates["fwd_d_fixed"].tolist()[0]+ (((df['Weight slab as per X (KG)'] - 0.5)*2.0)*(Rates['fwd_d_additional'].tolist()[0]))
    return price

  elif df["Delivery Zone as per X"] =="d" and df["Type of Shipment"] =="Forward and RTO charges":
    if df['Weight slab as per X (KG)']<=0.5:
      price = Rates['fwd_d_fixed'].tolist()[0]+Rates['rto_d_fixed'].tolist()[0]
    else:
      price = (Rates["fwd_d_fixed"].tolist()[0]+Rates['rto_d_fixed'].tolist()[0])+ (((df['Weight slab as per X (KG)'] - 0.5)*2.0)*(Rates['fwd_d_additional'].tolist()[0]))+(((df['Weight slab as per X (KG)'] - 0.5)*2.0)*(Rates['rto_d_additional'].tolist()[0]))
    return price
  
  if df["Delivery Zone as per X"] =="b" and df["Type of Shipment"] =="Forward charges":
    if df['Weight slab as per X (KG)']<=0.5:
      price = Rates['fwd_b_fixed'].tolist()[0]
    else:
      price = Rates["fwd_b_fixed"].tolist()[0]+ (((df['Weight slab as per X (KG)'] - 0.5)*2.0)*(Rates['fwd_b_additional'].tolist()[0]))
    return price

  elif df["Delivery Zone as per X"] =="b" and df["Type of Shipment"] =="Forward and RTO charges":
    if df['Weight slab as per X (KG)']<=0.5:
      price = Rates['fwd_b_fixed'].tolist()[0]+Rates['rto_b_fixed'].tolist()[0]
    else:
      price = (Rates["fwd_b_fixed"].tolist()[0]+Rates['rto_b_fixed'].tolist()[0])+ (((df['Weight slab as per X (KG)'] - 0.5)*2.0)*(Rates['fwd_b_additional'].tolist()[0]))+(((df['Weight slab as per X (KG)'] - 0.5)*2.0)*(Rates['rto_b_additional'].tolist()[0]))
    return price
  
  if df["Delivery Zone as per X"] =="e" and df["Type of Shipment"] =="Forward charges":
    if df['Weight slab as per X (KG)']<=0.5:
      price = Rates['fwd_e_fixed'].tolist()[0]
    else:
      price = Rates["fwd_e_fixed"].tolist()[0]+ (((df['Weight slab as per X (KG)'] - 0.5)*2.0)*(Rates['fwd_e_additional'].tolist()[0]))
    return price

  elif df["Delivery Zone as per X"] =="e" and df["Type of Shipment"] =="Forward and RTO charges":
    if df['Weight slab as per X (KG)']<=0.5:
      price = Rates['fwd_e_fixed'].tolist()[0]+Rates['rto_e_fixed'].tolist()[0]
    else:
      price = (Rates["fwd_e_fixed"].tolist()[0]+Rates['rto_e_fixed'].tolist()[0])+ (((df['Weight slab as per X (KG)'] - 0.5)*2.0)*(Rates['fwd_e_additional'].tolist()[0]))+(((df['Weight slab as per X (KG)'] - 0.5)*2.0)*(Rates['rto_e_additional'].tolist()[0]))
    return price
  
  if df["Delivery Zone as per X"] =="a" and df["Type of Shipment"] =="Forward charges":
    if df['Weight slab as per X (KG)']<=0.5:
      price = Rates['fwd_a_fixed'].tolist()[0]
    else:
      price = Rates["fwd_a_fixed"].tolist()[0]+ (((df['Weight slab as per X (KG)'] - 0.5)*2.0)*(Rates['fwd_a_additional'].tolist()[0]))
    return price

  elif df["Delivery Zone as per X"] =="a" and df["Type of Shipment"] =="Forward and RTO charges":
    if df['Weight slab as per X (KG)']<=0.5:
      price = Rates['fwd_a_fixed'].tolist()[0]+Rates['rto_a_fixed'].tolist()[0]
    else:
      price = (Rates["fwd_a_fixed"].tolist()[0]+Rates['rto_a_fixed'].tolist()[0])+ (((df['Weight slab as per X (KG)'] - 0.5)*2.0)*(Rates['fwd_a_additional'].tolist()[0]))+(((df['Weight slab as per X (KG)'] - 0.5)*2.0)*(Rates['rto_a_additional'].tolist()[0]))
    return 
  
  if df["Delivery Zone as per X"] =="c" and df["Type of Shipment"] =="Forward charges":
    if df['Weight slab as per X (KG)']<=0.5:
      price = Rates['fwd_c_fixed'].tolist()[0]
    else:
      price = Rates["fwd_c_fixed"].tolist()[0]+ (((df['Weight slab as per X (KG)'] - 0.5)*2.0)*(Rates['fwd_c_additional'].tolist()[0]))
    return price

  elif df["Delivery Zone as per X"] =="c" and df["Type of Shipment"] =="Forward and RTO charges":
    if df['Weight slab as per X (KG)']<=0.5:
      price = Rates['fwd_c_fixed'].tolist()[0]+Rates['rto_c_fixed'].tolist()[0]
    else:
      price = (Rates["fwd_c_fixed"].tolist()[0]+Rates['rto_c_fixed'].tolist()[0])+ (((df['Weight slab as per X (KG)'] - 0.5)*2.0)*(Rates['fwd_c_additional'].tolist()[0]))+(((df['Weight slab as per X (KG)'] - 0.5)*2.0)*(Rates['rto_c_additional'].tolist()[0]))
    return price

           
ExpectedResult['Expected Charge as per X (Rs.)'] = ExpectedResult.apply(charge,axis=1) #Expected Charge as per X Caculation 
ExpectedResult=ExpectedResult.round({"Expected Charge as per X (Rs.)":1})
ExpectedResult


,Order ID,Total weight as per X (G),Total weight as per X (KG),Weight slab as per X (KG),AWB Code,Total weight as per Courier Company (KG),Warehouse Pincode as per Courier Company,Customer Pincode as per Courier Company,Delivery Zone charged by Courier Company,Type of Shipment,Charges Billed by Courier Company (Rs.),Weight slab charged by Courier Company (KG)),Warehouse Pincode as per X,Customer Pincode as per X,Delivery Zone as per X,Expected Charge as per X (Rs.)
0,2001806210,220.0,0.2,0.5,1091117221940,2.92,121003,140604,b,Forward charges,174.5,3.0,121003,140604,b,33.0
1,2001806226,480.0,0.5,0.5,1091117222065,0.68,121003,723146,d,Forward charges,90.2,1.0,121003,723146,d,45.4
2,2001806229,500.0,0.5,0.5,1091117222080,0.71,121003,421204,d,Forward charges,90.2,1.0,121003,421204,d,45.4
3,2001806232,1302.0,1.3,1.5,1091117222124,1.30,121003,507101,d,Forward charges,135.0,1.5,121003,507101,d,135.0
4,2001806233,245.0,0.2,0.5,1091117222135,0.78,121003,263139,b,Forward charges,61.3,1.0,121003,263139,b,33.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119,2001821995,477.0,0.5,0.5,1091121183730,0.50,121003,342008,d,Forward charges,45.4,0.5,121003,342008,b,33.0
120,2001822466,1376.0,1.4,1.5,1091121305541,1.10,121003,342301,d,Forward charges,135.0,1.5,121003,342301,b,89.6
121,2001823564,672.0,0.7,1.0,1091121666133,0.70,121003,492001,d,Forward and RTO charges,172.8,1.0,121003,492001,d,176.3
122,2001825261,1557.0,1.6,2.0,1091121981575,1.60,121003,517128,d,Forward and RTO charges,345.0,2.0,121003,517128,d,355.5


In [ ]:
FinalResult = ExpectedResult.drop(["Total weight as per X (G)","Warehouse Pincode as per Courier Company","Customer Pincode as per Courier Company","Type of Shipment","Warehouse Pincode as per X","Customer Pincode as per X"],axis=1)
# Dropping columns to get the the Expected_Result in the given Format
FinalResult=FinalResult.iloc[:,[0,3,1,2,4,7,8,5,9,6]] #Rearranging the columns to get the Expected_Result in the given Format
FinalResult["Difference Between Expected Charges and Billed Charges (Rs.)"]= FinalResult["Expected Charge as per X (Rs.)"]-FinalResult["Charges Billed by Courier Company (Rs.)"] 
#Performing Calculation to get the Difference Between Expected Charges and Billed Charges (Rs.)
FinalResult

,Order ID,AWB Code,Total weight as per X (KG),Weight slab as per X (KG),Total weight as per Courier Company (KG),Weight slab charged by Courier Company (KG)),Delivery Zone as per X,Delivery Zone charged by Courier Company,Expected Charge as per X (Rs.),Charges Billed by Courier Company (Rs.),Difference Between Expected Charges and Billed Charges (Rs.)
0,2001806210,1091117221940,0.2,0.5,2.92,3.0,b,b,33.0,174.5,-141.5
1,2001806226,1091117222065,0.5,0.5,0.68,1.0,d,d,45.4,90.2,-44.8
2,2001806229,1091117222080,0.5,0.5,0.71,1.0,d,d,45.4,90.2,-44.8
3,2001806232,1091117222124,1.3,1.5,1.30,1.5,d,d,135.0,135.0,0.0
4,2001806233,1091117222135,0.2,0.5,0.78,1.0,b,b,33.0,61.3,-28.3
...,...,...,...,...,...,...,...,...,...,...,...
119,2001821995,1091121183730,0.5,0.5,0.50,0.5,b,d,33.0,45.4,-12.4
120,2001822466,1091121305541,1.4,1.5,1.10,1.5,b,d,89.6,135.0,-45.4
121,2001823564,1091121666133,0.7,1.0,0.70,1.0,d,d,176.3,172.8,3.5
122,2001825261,1091121981575,1.6,2.0,1.60,2.0,d,d,355.5,345.0,10.5


In [ ]:
FinalResult.to_excel("Expected_Result.xlsx") # Converting the FinalResult into excel file

In [ ]:
# Getting Count Values and Amount for the Summary Report according to the applied conditions
Count1=len(FinalResult[FinalResult["Difference Between Expected Charges and Billed Charges (Rs.)"]== 0])
Count2=len(FinalResult[FinalResult["Difference Between Expected Charges and Billed Charges (Rs.)"] < 0])
Count3=len(FinalResult[FinalResult["Difference Between Expected Charges and Billed Charges (Rs.)"] > 0])


Amount1=FinalResult.loc[FinalResult['Difference Between Expected Charges and Billed Charges (Rs.)'] == 0, 'Charges Billed by Courier Company (Rs.)'].sum()
Amount2=FinalResult.loc[FinalResult['Difference Between Expected Charges and Billed Charges (Rs.)'] < 0, 'Difference Between Expected Charges and Billed Charges (Rs.)'].sum()
Amount3=FinalResult.loc[FinalResult['Difference Between Expected Charges and Billed Charges (Rs.)'] > 0, 'Difference Between Expected Charges and Billed Charges (Rs.)'].sum()


In [ ]:
# Created a empty Summary Dataframe with columns Count and Amount and index of Total Orders - Correctly Charged,Total Orders - Over Charged and Total Orders - Under Charged
Summary = pd.DataFrame(columns=['Count','Amount'],index=['Total Orders - Correctly Charged',"Total Orders - Over Charged","Total Orders - Under Charged"])

Summary.loc['Total Orders - Correctly Charged'] = [Count1,Amount1] # append rows one by one to it using DataFrame.loc[] property.
Summary.loc['Total Orders - Over Charged'] = [Count2,Amount2]
Summary.loc['Total Orders - Under Charged'] = [Count3,Amount3]
Summary

,Count,Amount
Total Orders - Correctly Charged,26,2292.6
Total Orders - Over Charged,81,-4578.2
Total Orders - Under Charged,17,406.0


In [ ]:
Summary.to_excel("Summary.xlsx") # Converting the Summary into excel file